# Size Portfolio for CRSP Securities

In [2]:
import pandas as pd 
import numpy as np
import datetime as dt
import wrds
from dateutil.relativedelta import *

## Connect to WRDS 

In [3]:
conn = wrds.Connection(wrds_username='sevenz')

Loading library list...
Done


## Get CRSP Monthly Stocks for Decile Formation

In [4]:
msf = conn.raw_sql('''
                   SELECT a.permno, a.date,
                   a.ret, a.shrout, a.prc
                   FROM crsp.msf AS a 
                   WHERE a.date >= '12/01/1999'
                   ''', date_cols = ['date'])

### variable reference:

*permno: unique stock (share class) level identifier*

*(permco: unique company level identifier,
some company may have more than on class shares)*

*prc: Price*

*shrout: Number of Shares Outstanding*

*ret: Holding Period Return*

In [5]:
msf.describe()

,permno,ret,shrout,prc
count,1.741791e+06,1.688500e+06,1.732398e+06,1.698866e+06
mean,6.711130e+04,8.557388e-03,8.671068e+04,4.592137e+01
std,2.948650e+04,1.681236e-01,3.469439e+05,1.965336e+03
min,1.000100e+04,-9.936000e-01,1.000000e+00,-1.457500e+03
25%,4.700200e+04,-5.192497e-02,8.109000e+03,6.240000e+00
50%,8.130000e+04,4.533939e-03,2.320800e+04,1.570000e+01
75%,8.898400e+04,5.685578e-02,5.905800e+04,3.190000e+01
max,9.343600e+04,1.988359e+01,2.920640e+07,3.395900e+05


In [7]:
msf.head()

,permno,date,ret,shrout,prc
0,10001.0,1999-12-31,-0.004188,2450.0,-8.50000
1,10001.0,2000-01-31,-0.044118,2450.0,8.12500
2,10001.0,2000-02-29,0.015385,2450.0,8.25000
3,10001.0,2000-03-31,-0.015758,2464.0,-8.00000
4,10001.0,2000-04-28,0.011719,2464.0,-8.09375


### keep only records with non missing ret prc and shrout value

In [8]:
msf = msf[(msf['prc'].notna()) & (msf['ret'].notna()) & (msf['shrout'].notna())]
'''#alternatives:
msf = msf.dropna(subset = ['prc','ret','shrout'])
'''
msf.describe()

,permno,ret,shrout,prc
count,1.688500e+06,1.688500e+06,1.688500e+06,1.688500e+06
mean,6.724082e+04,8.557388e-03,8.803301e+04,4.609401e+01
std,2.939298e+04,1.681236e-01,3.508013e+05,1.971357e+03
min,1.000100e+04,-9.936000e-01,1.000000e+00,-1.457500e+03
25%,4.767700e+04,-5.192497e-02,8.246000e+03,6.230000e+00
50%,8.150100e+04,4.533939e-03,2.364300e+04,1.570000e+01
75%,8.900100e+04,5.685578e-02,6.003800e+04,3.197000e+01
max,9.343600e+04,1.988359e+01,2.920640e+07,3.395900e+05


In [9]:
msf['permno'] = msf['permno'].astype(int)
msf['size'] = msf['shrout'] * msf['prc'].abs()
msf['year'] = msf['date'].dt.year
msf['month'] = msf['date'].dt.month
msf.head()

,permno,date,ret,shrout,prc,size,year,month
0,10001,1999-12-31,-0.004188,2450.0,-8.50000,20825.00,1999,12
1,10001,2000-01-31,-0.044118,2450.0,8.12500,19906.25,2000,1
2,10001,2000-02-29,0.015385,2450.0,8.25000,20212.50,2000,2
3,10001,2000-03-31,-0.015758,2464.0,-8.00000,19712.00,2000,3
4,10001,2000-04-28,0.011719,2464.0,-8.09375,19943.00,2000,4


### create msf_dec

In [10]:
msf_dec = msf.query('month == 12')[['date','permno','year','size']]
msf_dec.head()

,date,permno,year,size
0,1999-12-31,10001,1999,20825.000000
12,2000-12-29,10001,2000,24355.500000
24,2001-12-31,10001,2001,29380.699511
36,2002-12-31,10001,2002,19046.440565
48,2003-12-31,10001,2003,15446.199505


### create msf_ls

In [11]:
msf_ls = msf.sort_values(['permno','date'])
msf_ls['year_prev'] = msf_ls['year']-1
msf_ls['size_lag'] = msf_ls.groupby('permno')['size'].shift(1)
msf_ls['size_lag'] = np.where(msf_ls['size_lag'].isna(),\
                             msf_ls['size']/(1+msf_ls['ret']), msf_ls['size_lag'])
'''
np.where(condition,x,y):
x if condition else y
'''
msf_ls.head()

,permno,date,ret,shrout,prc,size,year,month,year_prev,size_lag
0,10001,1999-12-31,-0.004188,2450.0,-8.50000,20825.00,1999,12,1998,20912.576138
1,10001,2000-01-31,-0.044118,2450.0,8.12500,19906.25,2000,1,1999,20825.000000
2,10001,2000-02-29,0.015385,2450.0,8.25000,20212.50,2000,2,1999,19906.250000
3,10001,2000-03-31,-0.015758,2464.0,-8.00000,19712.00,2000,3,1999,20212.500000
4,10001,2000-04-28,0.011719,2464.0,-8.09375,19943.00,2000,4,1999,19712.000000


## Compute Deciles for Each DEC 

In [12]:
msf_dec = msf_dec.sort_values(['year'])
msf_dec['decile'] = 1+msf_dec.groupby('year')['size']\
.transform(lambda x: pd.qcut(x, 10, labels=False))
msf_dec.head()

,date,permno,year,size,decile
0,1999-12-31,10001,1999,20825.000,2.0
803619,1999-12-31,80001,1999,135171.000,5.0
803491,1999-12-31,80000,1999,59602.500,4.0
803325,1999-12-31,79996,1999,3037724.875,10.0
803211,1999-12-31,79994,1999,7507.500,1.0


[*pd.qcut( )*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html)

## Assign Size Group to All Months

In [13]:
msf_groups = pd.merge(msf_ls[['permno','date','ret','size_lag','year_prev']],\
                      msf_dec[['permno','year','decile']],how = 'left', \
                      left_on = ['permno','year_prev'], right_on = ['permno','year'])
msf_groups = msf_groups.dropna(subset = ['decile'])
msf_groups.head()

,permno,date,ret,size_lag,year_prev,year,decile
1,10001,2000-01-31,-0.044118,20825.00,1999,1999.0,2.0
2,10001,2000-02-29,0.015385,19906.25,1999,1999.0,2.0
3,10001,2000-03-31,-0.015758,20212.50,1999,1999.0,2.0
4,10001,2000-04-28,0.011719,19712.00,1999,1999.0,2.0
5,10001,2000-05-31,-0.023166,19943.00,1999,1999.0,2.0


## Compute Size Weighted Returns

In [14]:
msf_groups = msf_groups.sort_values(['decile','date'])

### function to calculate value weighted return

In [15]:
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d*w).sum()/w.sum()
    except ZeroDivisionError:
        return np.nan

### value-weighted return

In [18]:
vwrets = msf_groups.groupby(['decile','date']).apply(wavg, 'ret','size_lag')\
.to_frame().reset_index().rename(columns = {0: 'vwrt'})
vwrets.head()

,decile,date,vwrt
0,1.0,2000-01-31,0.222626
1,1.0,2000-02-29,0.246672
2,1.0,2000-03-31,-0.024419
3,1.0,2000-04-28,-0.175907
4,1.0,2000-05-31,-0.093232


## Compare Results with CRSP MSIX

In [20]:
msix = conn.raw_sql("""
                    SELECT caldt, decret1, decret2, decret3, decret4, decret5,
                    decret6, decret7, decret8, decret9, decret10
                    FROM crsp.msix where caldt >= '12/01/1999'
                    """, date_cols=['caldt'])

### transpose msix data

In [27]:
msix1 = pd.melt(msix, id_vars = 'caldt',\
               value_vars = ['decret1', 'decret2', 'decret3', 'decret4', 'decret5',
                    'decret6', 'decret7', 'decret8', 'decret9', 'decret10'])

### extract decile information from decret

In [28]:
msix1['decile'] = msix1['variable'].str[6:].astype(int)

### rename return column

In [29]:
msix1 = msix1.rename(columns={'value':'decret','caldt':'date'})
msix1 = msix1.drop(['variable'], axis = 1)

decile_returns = pd.merge(vwrets, msix1, how='left', on=['date','decile'])

## End of Program